In [3]:
# FETCH HISTORICAL DATA

from torchtrade.data.cdd import CryptoDataDownload
import pandas as pd

cdd = CryptoDataDownload()

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
], axis=1)

bitstamp_data

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,5.599300e+02
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,2.735800e+02
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,9.177900e+02
3,2018-05-15 09:00:00,1526374800,8728.49,8754.40,8701.35,8708.32,1.826200e+02
4,2018-05-15 10:00:00,1526378400,8708.32,8865.00,8695.11,8795.90,1.260690e+03
...,...,...,...,...,...,...,...
57494,2024-12-04 20:00:00,1733342400,97381.00,99250.00,97296.00,99023.00,8.397736e+07
57495,2024-12-04 21:00:00,1733346000,99025.00,99165.00,97651.00,97877.00,4.844984e+07
57496,2024-12-04 22:00:00,1733349600,97877.00,98742.00,97727.00,98351.00,3.571891e+07
57497,2024-12-04 23:00:00,1733353200,98353.00,99163.00,98353.00,98773.00,1.555292e+07


In [4]:
# DEFINE EXCHANGES

from torchtrade.oms.exchanges import Exchange
from torchtrade.oms.services.execution.simulated import execute_order
from torchtrade.feed.core.base import Stream


bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC")
)